In [7]:


import os
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam
from PIL import Image
from torchvision import transforms

# Yahan pe dataset loader define ho raha hai
class BUSIDataset:
    def __init__(self, root):
        self.root = root
        # Yahan pe classes ko identify kar rahe hain
        self.classes = [cls for cls in os.listdir(root) if os.path.isdir(os.path.join(root, cls))]
        # Yahan pe images ke liye transformation define ho raha hai
        self.transform = transforms.Compose([
            transforms.Resize((224, 224)),  # images ko resize kar rahe hain
            transforms.ToTensor()  # images ko tensor me convert kar rahe hain
        ])
        self.images = {}
        for cls in self.classes:
            images = os.listdir(os.path.join(root, cls))  # har class ke images read kar rahe hain
            self.images[cls] = [os.path.join(root, cls, img) for img in images]  # full path store kar rahe hain

    # Yahan pe ek few-shot episode create ho raha hai
    def get_episode(self, n_way=3, k_shot=5, q_query=5):
        selected_classes = random.sample(self.classes, n_way)  # randomly classes choose kar rahe hain
        support, query = [], []

        for label, cls in enumerate(selected_classes):
            selected_images = random.sample(self.images[cls], k_shot + q_query)  # support + query images select
            support += [(img, label) for img in selected_images[:k_shot]]  # support images
            query   += [(img, label) for img in selected_images[k_shot:]]  # query images

        def load(samples):
            X, y = [], []
            for img_path, label in samples:
                img = Image.open(img_path).convert("RGB")  # image open kar rahe hain aur RGB me convert kar rahe hain
                img = self.transform(img)  # transformation apply kar rahe hain
                X.append(img)
                y.append(label)
            return torch.stack(X), torch.tensor(y)  # support/query tensors return kar rahe hain

        support_x, support_y = load(support)  # support images load ho rahe hain
        query_x, query_y = load(query)  # query images load ho rahe hain
        return support_x, support_y, query_x, query_y

# Yahan pe ProtoNet model define ho raha hai
class ProtoNet(nn.Module):
    def __init__(self):
        super().__init__()
        # Yahan pe CNN encoder define ho raha hai
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 64, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2),  # convolution + activation + pooling
            nn.Conv2d(64, 64, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(64, 64, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Flatten()  # flatten kar ke embedding nikal rahe hain
        )

    def forward(self, x):
        return self.encoder(x)  # forward pass se embedding nikal rahe hain

# Yahan pe training loop define ho raha hai
def train_protonet(dataset, epochs=30, n_way=3, k_shot=5, q_query=5):
    model = ProtoNet()  # model initialize
    optimizer = Adam(model.parameters(), lr=0.001)  # optimizer define ho raha hai

    for epoch in range(epochs):
        # Yahan pe single episode generate ho raha hai (base paper style)
        support_x, support_y, query_x, query_y = dataset.get_episode(n_way, k_shot, q_query)

        support_embed = model(support_x)  # support images embeddings nikal rahe hain
        query_embed = model(query_x)  # query images embeddings nikal rahe hain

        # Yahan pe prototypes calculate ho rahe hain
        prototypes = []
        for cls in range(n_way):
            prototypes.append(support_embed[support_y == cls].mean(dim=0))  # har class ka prototype
        prototypes = torch.stack(prototypes)

        # Yahan pe distances aur predictions calculate ho rahe hain
        distances = torch.cdist(query_embed, prototypes)  # Euclidean distance
        predictions = distances.argmin(dim=1)  # nearest prototype select

        # Yahan pe loss aur accuracy calculate ho rahi hai
        loss = F.cross_entropy(-distances, query_y)  # loss function
        acc = (predictions == query_y).float().mean()  # accuracy

        optimizer.zero_grad()  # gradients reset ho rahe hain
        loss.backward()  # backpropagation ho raha hai
        optimizer.step()  # weights update ho rahe hain

        # Yahan pe epoch ka result print ho raha hai
        print(f"Epoch {epoch+1}/{epochs} | Loss: {loss.item():.4f} | Acc: {acc.item()*100:.2f}%")

    return model  # trained model return ho raha hai

# Yahan pe training run ho rahi hai
if __name__ == "__main__":
    dataset_path = r"C:\Users\THE DEVICE MASTER\Desktop\archive\Dataset_BUSI_with_GT"  # dataset path
    dataset = BUSIDataset(dataset_path)  # dataset load ho raha hai
    model = train_protonet(dataset, epochs=30, n_way=3, k_shot=5, q_query=5)  # model train ho raha hai


Epoch 1/30 | Loss: 1.0046 | Acc: 46.67%
Epoch 2/30 | Loss: 1.8541 | Acc: 33.33%
Epoch 3/30 | Loss: 0.8948 | Acc: 66.67%
Epoch 4/30 | Loss: 1.0469 | Acc: 33.33%
Epoch 5/30 | Loss: 1.0267 | Acc: 33.33%
Epoch 6/30 | Loss: 0.7865 | Acc: 60.00%
Epoch 7/30 | Loss: 1.0747 | Acc: 46.67%
Epoch 8/30 | Loss: 1.1528 | Acc: 26.67%
Epoch 9/30 | Loss: 0.9010 | Acc: 66.67%
Epoch 10/30 | Loss: 1.0965 | Acc: 46.67%
Epoch 11/30 | Loss: 1.0171 | Acc: 46.67%
Epoch 12/30 | Loss: 1.1063 | Acc: 46.67%
Epoch 13/30 | Loss: 0.9012 | Acc: 66.67%
Epoch 14/30 | Loss: 1.0785 | Acc: 40.00%
Epoch 15/30 | Loss: 1.0213 | Acc: 46.67%
Epoch 16/30 | Loss: 0.9014 | Acc: 60.00%
Epoch 17/30 | Loss: 0.8534 | Acc: 46.67%
Epoch 18/30 | Loss: 1.2167 | Acc: 46.67%
Epoch 19/30 | Loss: 1.1818 | Acc: 26.67%
Epoch 20/30 | Loss: 1.6382 | Acc: 40.00%
Epoch 21/30 | Loss: 1.0780 | Acc: 40.00%
Epoch 22/30 | Loss: 1.9810 | Acc: 6.67%
Epoch 23/30 | Loss: 1.3081 | Acc: 13.33%
Epoch 24/30 | Loss: 1.1373 | Acc: 40.00%
Epoch 25/30 | Loss: 1.0546

In [4]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu
import torch
print(torch.__version__)
print(torch.cuda.is_available())  # Should print False since CPU-only




^C
2.9.1+cpu
False
Looking in indexes: https://download.pytorch.org/whl/cpu
   ---------------------------------------- 0.0/110.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/110.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/110.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/110.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/110.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/110.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/110.9 MB ? eta -:--:--
   ---------------------------------------- 0.3/110.9 MB ? eta -:--:--
   ---------------------------------------- 0.3/110.9 MB ? eta -:--:--
   ---------------------------------------- 0.3/110.9 MB ? eta -:--:--
   ---------------------------------------- 0.3/110.9 MB ? eta -:--:--
   ---------------------------------------- 0.3/110.9 MB ? eta -:--:--
   ---------------------------------------- 0.3/110.9 MB ? eta -:--:--
 